VAE - Pawsitive Aging
==============
The following code was heavily based upon the Keras VAE implmentation code with changes to the architecture and optimizations 

In [4]:
# library includes
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

In [5]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [6]:
# data loading from CSV
data_path = "./data_sources/"
processedDogs_path = "./data_sources/images/processed_dogs/"
data_csv_path = "./data_sources/dog_data.csv"

dogCSV = pd.read_csv(data_csv_path)

dogImagesPaths = []
for fileDirectory in dogCSV["fileDirectory"]:
    dogImagesPaths.append(processedDogs_path + fileDirectory)

dogCSV["fullFilePath"] = dogImagesPaths

# removing all the rows that are labeled other than "middle-aged", "old", "young"
properLabels = ["old", "young", "middle-aged"]
dogCSV = dogCSV[dogCSV["label"].isin(properLabels)]
dogCSV

,index,fileDirectory,label,fullFilePath
0,0,dog000000.jpg,middle-aged,./data_sources/images/processed_dogs/dog000000...
1,1,dog000001.jpg,middle-aged,./data_sources/images/processed_dogs/dog000001...
2,2,dog000002.jpg,middle-aged,./data_sources/images/processed_dogs/dog000002...
3,3,dog000003.jpg,old,./data_sources/images/processed_dogs/dog000003...
4,4,dog000004.jpg,middle-aged,./data_sources/images/processed_dogs/dog000004...
...,...,...,...,...
5570,5570,dog005570.jpg,young,./data_sources/images/processed_dogs/dog005570...
5571,5571,dog005571.jpg,middle-aged,./data_sources/images/processed_dogs/dog005571...
5572,5572,dog005572.jpg,middle-aged,./data_sources/images/processed_dogs/dog005572...
5573,5573,dog005573.jpg,young,./data_sources/images/processed_dogs/dog005573...


In [7]:
# data preprocessing
loaded_images = []
r_data = []
g_data = []
b_data = []

for imageFilePath in dogCSV["fullFilePath"]:
    img = load_img(imageFilePath, target_size=(150, 150), color_mode='rgb')
    img_array = img_to_array(img) / 255.0
    imageData = np.array(img_array)
    loaded_images.append(imageData)
    r_data.append(np.array(imageData[:, :, 0]).flatten())
    g_data.append(np.array(imageData[:, :, 1]).flatten())
    b_data.append(np.array(imageData[:, :, 2]).flatten())    
    
dogCSV["RGB_data"] = loaded_images
dogCSV["R_data"] = r_data
dogCSV["G_data"] = g_data
dogCSV["B_data"] = b_data

dogCSV

,index,fileDirectory,label,fullFilePath,RGB_data,R_data,G_data,B_data
0,0,dog000000.jpg,middle-aged,./data_sources/images/processed_dogs/dog000000...,"[[[0.49411765, 0.43137255, 0.43529412], [0.501...","[0.49411765, 0.5019608, 0.39607844, 0.43137255...","[0.43137255, 0.4392157, 0.33333334, 0.36862746...","[0.43529412, 0.44313726, 0.3372549, 0.37254903..."
1,1,dog000001.jpg,middle-aged,./data_sources/images/processed_dogs/dog000001...,"[[[0.28235295, 0.39215687, 0.4862745], [0.3098...","[0.28235295, 0.30980393, 0.30980393, 0.2784314...","[0.39215687, 0.41960785, 0.41960785, 0.3882353...","[0.4862745, 0.5137255, 0.5137255, 0.48235294, ..."
2,2,dog000002.jpg,middle-aged,./data_sources/images/processed_dogs/dog000002...,"[[[0.32156864, 0.078431375, 0.13725491], [0.32...","[0.32156864, 0.32156864, 0.32156864, 0.3098039...","[0.078431375, 0.078431375, 0.07058824, 0.05882...","[0.13725491, 0.13725491, 0.13333334, 0.1215686..."
3,3,dog000003.jpg,old,./data_sources/images/processed_dogs/dog000003...,"[[[1.0, 0.99607843, 1.0], [0.9529412, 0.933333...","[1.0, 0.9529412, 1.0, 0.9490196, 0.9843137, 1....","[0.99607843, 0.93333334, 0.99215686, 0.9137255...","[1.0, 0.9490196, 1.0, 0.93333334, 0.95686275, ..."
4,4,dog000004.jpg,middle-aged,./data_sources/images/processed_dogs/dog000004...,"[[[0.011764706, 0.011764706, 0.011764706], [0....","[0.011764706, 0.011764706, 0.011764706, 0.0078...","[0.011764706, 0.011764706, 0.011764706, 0.0078...","[0.011764706, 0.011764706, 0.011764706, 0.0078..."
...,...,...,...,...,...,...,...,...
5570,5570,dog005570.jpg,young,./data_sources/images/processed_dogs/dog005570...,"[[[0.39607844, 0.40392157, 0.4], [0.39607844, ...","[0.39607844, 0.39607844, 0.4, 0.4, 0.40392157,...","[0.40392157, 0.40392157, 0.40784314, 0.4078431...","[0.4, 0.4, 0.40392157, 0.40392157, 0.40784314,..."
5571,5571,dog005571.jpg,middle-aged,./data_sources/images/processed_dogs/dog005571...,"[[[0.023529412, 0.023529412, 0.023529412], [0....","[0.023529412, 0.023529412, 0.023529412, 0.0235...","[0.023529412, 0.023529412, 0.023529412, 0.0235...","[0.023529412, 0.023529412, 0.023529412, 0.0235..."
5572,5572,dog005572.jpg,middle-aged,./data_sources/images/processed_dogs/dog005572...,"[[[0.6117647, 0.39607844, 0.1764706], [0.63137...","[0.6117647, 0.6313726, 0.72156864, 0.43529412,...","[0.39607844, 0.40392157, 0.50980395, 0.3058823...","[0.1764706, 0.21568628, 0.36078432, 0.18431373..."
5573,5573,dog005573.jpg,young,./data_sources/images/processed_dogs/dog005573...,"[[[0.007843138, 0.007843138, 0.007843138], [0....","[0.007843138, 0.003921569, 0.003921569, 0.0039...","[0.007843138, 0.003921569, 0.003921569, 0.0039...","[0.007843138, 0.003921569, 0.003921569, 0.0039..."


In [45]:
# splitting the test set and training set (25% test set, 75% training set)
imagesData = dogCSV["RGB_data"]
imageLabels = dogCSV["label"]

imageArr = []
for imageData in imagesData.to_list():
    imageArr.append(imageData)
X_train_tensor = tf.convert_to_tensor(imageArr)

In [47]:
X_train_tensor.shape

TensorShape([5453, 150, 150, 3])

In [26]:
# enumerating and mapping the labels to values
labelDictionary = {"middled-aged":0, "old":1, "young":2}

In [22]:
# defining the encoder
latent_dim = 10
inputShape = (150, 150, 3)

encoder_inputs = keras.Input(shape=inputShape)
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=3, padding="same")(x)
x = layers.Conv2D(128, 5, activation="relu", strides=1, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)   │ (None, 150, 150, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_9 (Conv2D)             │ (None, 75, 75, 32)        │             896 │ input_layer_12[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_10 (Conv2D)            │ (None, 25, 25, 64)        │          18,496 │ conv2d_9[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_11 (Conv2D)            │ (None, 25, 25, 128)       │         204,928 │ conv2d_10[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_3 (Flatten)           │ (None, 80000)             │               0 │ conv2d_11[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_12 (Dense)              │ (None, 128)               │      10,240,128 │ flatten_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_13 (Dense)              │ (None, 64)                │           8,256 │ dense_12[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ z_mean (Dense)                │ (None, 10)                │             650 │ dense_13[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ z_log_var (Dense)             │ (None, 10)                │             650 │ dense_13[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sampling_3 (Sampling)         │ (None, 10)                │               0 │ z_mean[0][0],              │
│                               │                           │                 │ z_log_var[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 10,474,004 (39.96 MB)

 Trainable params: 10,474,004 (39.96 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# defining the decoder
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(25 * 25 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((25, 25, 64))(x)
x = layers.Conv2DTranspose(128, 5, activation="relu", strides=1, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=3, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 40000)               │         440,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_9 (Reshape)                  │ (None, 25, 25, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_29                  │ (None, 25, 25, 128)         │         204,928 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_30                  │ (None, 75, 75, 64)          │          73,792 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_31                  │ (None, 150, 150, 32)        │          18,464 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_32                  │ (None, 150, 150, 3)         │             867 │
│ (Conv2DTranspose)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 738,051 (2.82 MB)

 Trainable params: 738,051 (2.82 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# defining the VAE class
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self):
        

In [51]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(X_train_tensor, epochs=30, batch_size=128)

Epoch 1/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 36s 704ms/step - kl_loss: 4.9218 - loss: 15517.2959 - reconstruction_loss: 15511.5518 - total_loss: 15481.1006
Epoch 2/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 33s 679ms/step - kl_loss: 1.9781 - loss: 15475.0605 - reconstruction_loss: 15471.4844 - total_loss: 15404.6875
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 29s 685ms/step - kl_loss: 34.7952 - loss: 14947.3252 - reconstruction_loss: 14910.5557 - total_loss: 14860.3936
Epoch 4/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 30s 687ms/step - kl_loss: 39.9394 - loss: 14502.5742 - reconstruction_loss: 14461.5518 - total_loss: 14454.8164
Epoch 5/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 30s 693ms/step - kl_loss: 36.1860 - loss: 14244.6465 - reconstruction_loss: 14206.9609 - total_loss: 14178.6436
Epoch 6/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 30s 685ms/step - kl_loss: 33.9436 - loss: 14063.9277 - reconstruction_loss: 14029.9951 - total_loss: 14064.4551
Epoch 7/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 30s 698ms/step - kl_loss: 33.6880 - loss: 14001.9863 - reconstru